In [1]:
import numpy as np 
from PIL import Image  
import cv2  
import matplotlib.pyplot as plt  
import pickle  
from matplotlib import style  
import time 
import math

from func import *

style.use("ggplot")  # setting style

In [2]:
SIZE = 20

EPISODES = 25000
MOVE_PENALTY = 1  
NFZ_PENALTY = 300  
POI_REWARD = 25
epsilon = 0.5 
EPS_DECAY = 0.9999 
SHOW_EVERY = 1000
EPISODE_STEPS=200

# start_q_table = "qtable-1613111805.pickle"
start_q_table = None


LEARNING_RATE = 0.1
DISCOUNT = 0.95

ACTIONS = 4
AGENTS = 3
PoIs = 20
NFZs = 5

Environment_grid = np.zeros((SIZE, SIZE, 3), dtype=np.uint8) #size*size grid with 3 values at each point to represent color
Occupied = np.zeros((SIZE, SIZE), dtype=np.uint8)

UAV_P = 1  
POI_P = 2  
NFZ_P = 3
CUR_P = 4
VSTD_P = 5



d = {1: (255, 175, 0),  # blueish color
     2: (0, 255, 0),  # green
     3: (0, 0, 255),   # red
     4: (0, 120, 0),   # light green
     5: (255,255,255)}  # white

In [3]:
class OBJECT:
    def __init__(self):
        x=0
        y=0
        while(1):
            a = np.random.randint(0, SIZE)
            b = np.random.randint(0, SIZE)
            if (a,b) in Uniqe_points:
                continue
            else:
                Uniqe_points.append((a,b))
                x=a
                y=b
                break
        self.x = x
        self.y = y
        self.goals = []
        self.visited = []
        self.val=-1
        self.idx=-1
    def __str__(self):
        return f"{self.x}, {self.y}"
    def __sub__(self, other):
        return (self.x-other.x, self.y-other.y)
    
    def action(self, choice):
        '''
        Gives us 8 total movement options. (0,1,2,3,4,5,6,7)
        '''
        if choice == 0:
            self.move(x=1, y=1)
        elif choice == 1:
            self.move(x=-1, y=-1)
        elif choice == 2:
            self.move(x=-1, y=1)
        elif choice == 3:
            self.move(x=1, y=-1)
        elif choice == 4:
            self.move(x=0, y=-1)
        elif choice == 5:
            self.move(x=0, y=1)
        elif choice == 6:
            self.move(x=1, y=0)
        elif choice == 7:
            self.move(x=-1, y=0)
            
    def move(self, x=False, y=False):

        # If no value for x, move randomly
        if not x:
            self.x += np.random.randint(-1, 2)
        else:
            self.x += x

        # If no value for y, move randomly
        if not y:
            self.y += np.random.randint(-1, 2)
        else:
            self.y += y


        # If we are out of bounds, fix!
        if self.x < 0:
            self.x = 0
        elif self.x > SIZE-1:
            self.x = SIZE-1
        if self.y < 0:
            self.y = 0
        elif self.y > SIZE-1:
            self.y = SIZE-1

In [4]:
if start_q_table is None:
    # initialize the q-table#
    q_table = {}
    for i in range(-SIZE+1, SIZE):
        for ii in range(-SIZE+1, SIZE):
            for iii in range(-SIZE+1, SIZE):
                    for iiii in range(-SIZE+1, SIZE):
                        q_table[((i, ii), (iii, iiii))] = [np.random.uniform(-5, 0) for i in range(8)]
else:
    with open(start_q_table, "rb") as f:
        q_table = pickle.load(f)

In [5]:
def initialize_eco(AGENTS, PoIs, NFZs):
    Agent_list = []
    POI_list = []
    NFZ_list = []
    for i in range(AGENTS):
        agent = OBJECT()
        Agent_list.append(agent)
    
    for i in range(PoIs):
        poi = OBJECT()
        POI_list.append(poi)

    for i in range(NFZs):
        nfz = OBJECT()
        NFZ_list.append(nfz)
        
    for i in range(AGENTS):
        curr_obj = OBJECT()
        curr_obj.x = -1
        curr_obj.y = -1
        current_PoI_list.append(curr_obj)
        current_NFZ.append(curr_obj)

    return Agent_list, POI_list, NFZ_list, current_PoI_list, current_NFZ

In [6]:
episode_rewards = []

MAX_VISITED = 0

show_it = []
rewards = []

for episode in range(EPISODES):

    episode_reward = 0
    Uniqe_points = []
    
    
    Agent_list = []
    POI_list = []
    NFZ_list = []
    current_PoI_list =[]
    current_NFZ = []

    Agent_list, POI_list, NFZ_list, current_PoI_list, current_NFZ = initialize_eco(AGENTS, PoIs, NFZs)
    
    reward_each_agent =[0]*len(Agent_list)
    
    paths_of_all_agents =[]

    for i in range(len(Agent_list)):
        a=[]
        paths_of_all_agents.append(a)


    # assign each point to UAVs in round robin manner
    for j in range(len(POI_list)):
        i = j%len(Agent_list)
        Agent_list[i].goals.append((POI_list[j].x,POI_list[j].y))
        Agent_list[i].visited.append(0)

    # calculate the distance from each assigned point
    for i in range(len(Agent_list)):

        assign_weights(Agent_list[i])

    # buy and sell points according to the close-ness of the point
    for j in range(len(POI_list)):
        clst_idx, clst_val = calculate_distance(Agent_list,POI_list[j])
        i = j%len(Agent_list)

        a = 0
        Agent_list[i].goals.pop(a)
        Agent_list[i].visited.pop(a)

        Agent_list[clst_idx].goals.append((POI_list[j].x,POI_list[j].y))
        Agent_list[clst_idx].visited.append(clst_val)
        
        if i!=clst_idx:
            reward_each_agent[i] += 10
            reward_each_agent[clst_idx] -= 10        

   
    # observe the new distribution of points and set the current target for each of the UAVs
    for i in range(len(Agent_list)):

        assign_weights(Agent_list[i])

        if Agent_list[i].goals:
            val, idx = min((val, idx) for (idx, val) in enumerate(Agent_list[i].visited))

            Agent_list[i].val = val
            Agent_list[i].idx = idx

            current_PoI_list[i].x = Agent_list[i].goals[idx][0]
            current_PoI_list[i].y = Agent_list[i].goals[idx][1]


    
    if episode % SHOW_EVERY == 0:
        show = True
    else:
        show = False
        


    for i in range(EPISODE_STEPS):
        
        ####################################################
        for agnt_i in range(len(Agent_list)):
            
            paths_of_all_agents[agnt_i].append((Agent_list[agnt_i].x,Agent_list[agnt_i].y))
            
            if current_PoI_list[agnt_i].x != -1:
                obs = (Agent_list[agnt_i]-current_PoI_list[agnt_i], NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE)) 
                if np.random.random() > epsilon:
                    # GET THE ACTION
                    action = np.argmax(q_table[obs])
                else:
                    action = np.random.randint(0, 4)

                # Taking selected action
                Agent_list[agnt_i].action(action)


                if Agent_list[agnt_i].x == NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE)[0] and Agent_list[agnt_i].y == NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE)[1]:
                    reward_each_agent[agnt_i] = -NFZ_PENALTY
                elif Agent_list[agnt_i].x == current_PoI_list[agnt_i].x and Agent_list[agnt_i].y == current_PoI_list[agnt_i].y:
                    reward_each_agent[agnt_i] = POI_REWARD
                else:
                    reward_each_agent[agnt_i] = -MOVE_PENALTY


                new_obs = (Agent_list[agnt_i]-current_PoI_list[agnt_i], NFZ_distance(Agent_list[agnt_i],NFZ_list,SIZE))  # new observation
                max_future_q = np.max(q_table[new_obs])  # max Q value for this new obs
                current_q = q_table[obs][action]  # current Q for our chosen action

                if reward_each_agent[agnt_i] == POI_REWARD:
                    new_q = POI_REWARD
                elif reward_each_agent[agnt_i] == -NFZ_PENALTY:
                    new_q = -NFZ_PENALTY
                else:
                    new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward_each_agent[agnt_i] + DISCOUNT * max_future_q)


                q_table[obs][action] = new_q
            else:
                break
        
        #############################################################
        
        if show:
            env = np.zeros((SIZE, SIZE, 3), dtype=np.uint8)  # starts an rbg of our size
            
            
            
            for i in range(len(POI_list)):
                env[POI_list[i].x][POI_list[i].y] = d[POI_P] # sets the Point of interest tile to green color
            
            for agnt_i in range(len(Agent_list)):
                if current_PoI_list[agnt_i].x != -1:
                    env[current_PoI_list[agnt_i].x][current_PoI_list[agnt_i].y] = d[CUR_P]  # sets the current Point of interest tile to light green color
                
                for i,v in enumerate(Agent_list[agnt_i].visited):
                    if v==1000:
                        env[Agent_list[agnt_i].goals[i][0]][Agent_list[agnt_i].goals[i][1]] = d[VSTD_P] # sets visited point to white           
                
                
                env[Agent_list[agnt_i].x][Agent_list[agnt_i].y] = d[UAV_P]  # sets the agent tile to blue
            
            for k in range(len(NFZ_list)):
                env[NFZ_list[k].x][NFZ_list[k].y] = d[NFZ_P]  # sets the no fly location to red
            img = Image.fromarray(env, 'RGB')  # reading to rgb
            img = img.resize((300, 300), resample=Image.BOX)  
            cv2.imshow("Eco Q", np.array(img))
            
            
            for agnt_i in range(len(Agent_list)):
                if reward_each_agent[agnt_i] == POI_REWARD or reward_each_agent[agnt_i] == -NFZ_PENALTY:  # crummy code to hang at the end if we reach abrupt end for good reasons or not.
                    if reward_each_agent[agnt_i] == POI_REWARD:
                            Agent_list[agnt_i].visited[Agent_list[agnt_i].idx] = 1000
                            

                            val, idx = min((val, idx) for (idx, val) in enumerate(Agent_list[agnt_i].visited))

                            if val==1000:
                                print("Agent ", agnt_i, "Completed : ", Agent_list[agnt_i].visited)
                                break

                            print("Agent ", agnt_i, "Visited : ", Agent_list[agnt_i].visited)
                            Agent_list[agnt_i].val = val
                            Agent_list[agnt_i].idx = idx


                            current_PoI_list[agnt_i].x = Agent_list[agnt_i].goals[idx][0]
                            current_PoI_list[agnt_i].y = Agent_list[agnt_i].goals[idx][1]
                            continue

                    if cv2.waitKey(500) & 0xFF == ord('q'):
                        break
                else:
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break

                    
        episode_reward += reward_each_agent[agnt_i] 
        for agnt_i in range(len(Agent_list)):
            if reward_each_agent[agnt_i] == POI_REWARD or reward_each_agent[agnt_i] == -NFZ_PENALTY:
                if reward_each_agent[agnt_i] == POI_REWARD:
                    Agent_list[agnt_i].visited[Agent_list[agnt_i].idx] = 1000
                    val, idx = min((val, idx) for (idx, val) in enumerate(Agent_list[agnt_i].visited))

                    if val==1000:
                        break

                    Agent_list[agnt_i].val = val
                    Agent_list[agnt_i].idx = idx


                    current_PoI_list[agnt_i].x = Agent_list[agnt_i].goals[idx][0]
                    current_PoI_list[agnt_i].y = Agent_list[agnt_i].goals[idx][1]
                    continue
                break
                
    
    p=0           
    for agent_i in range(len(Agent_list)):
        for k in range(len(Agent_list[agent_i].visited)):
            p += Agent_list[agent_i].visited[k]
            
    if p>MAX_VISITED:
        MAX_VISITED=p
        show_it=paths_of_all_agents
        temp = []
        for i in range(len(Agent_list)):
            temp.append(Agent_list[i].goals)
        
        rewards = temp
    episode_rewards.append(episode_reward)
    epsilon *= EPS_DECAY

    
cv2.destroyAllWindows()
moving_avg = np.convolve(episode_rewards, np.ones((SHOW_EVERY,))/SHOW_EVERY, mode='valid')


with open(f"qtable-{int(time.time())}.pickle", "wb") as f:
    pickle.dump(q_table, f)

Agent  2 Visited :  [1000, 3.605551275463989]
Agent  1 Visited :  [12.083045973594572, 16.97056274847714, 6.4031242374328485, 6.0, 10.63014581273465, 9.219544457292887, 8.246211251235321, 7.0710678118654755, 1000, 7.810249675906654, 12.649110640673518, 10.816653826391969, 12.806248474865697]
Agent  0 Visited :  [1000, 4.0, 2.0, 6.4031242374328485, 6.708203932499369, 3.0, 1.4142135623730951, 11.40175425099138, 3.0, 4.123105625617661, 4.47213595499958]
Agent  0 Visited :  [1000, 4.0, 1000, 6.4031242374328485, 6.708203932499369, 3.0, 1000, 11.40175425099138, 3.0, 4.123105625617661, 4.47213595499958]
Agent  1 Visited :  [1000, 3.0]
Agent  1 Completed :  [1000, 1000]
Agent  1 Completed :  [1000, 1000]
Agent  1 Completed :  [1000, 1000]
Agent  1 Completed :  [1000, 1000]
Agent  1 Completed :  [1000, 1000]
Agent  2 Visited :  [12.649110640673518, 4.47213595499958, 1000, 5.385164807134504]
Agent  0 Visited :  [2.23606797749979, 7.810249675906654, 6.082762530298219, 9.848857801796104, 1000, 3.6

Agent  0 Visited :  [1000, 1000, 1000, 13.038404810405298, 1000, 1000, 10.816653826391969]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000]

Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000]
A

Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agen

Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000]
Agent  0 Visited :  [6.4031242374328485, 4.0, 1000, 1.4142135623730951, 5.0, 7.615773105863909, 8.0, 2.23606797749979]
Agent  0 Visited :  [6.4031242374328485, 4.0, 1000, 1000, 5.0, 7.615773105863909, 8.0, 1000]
Agent  2 Visited :  [3.0, 1000, 7.0710678118654755, 8.94427190999916, 5.830951894845301, 11.180339887498949]
Agent  1 Visited :  [5.830951894845301, 9.219544457292887, 1000, 10.44030650891055, 3.1622776601683795, 4.123105625617661]
Agent  0 Visited :  [6.4031242374328485, 1000, 1000, 1000, 1000, 7.615773105863909, 8.0, 1000]
Agent  1 Visited :  [5.830951894845301, 9.219544457292887, 1000, 10.44030650891055, 1000, 1000]
Agent  0 Visited :  [1000, 1000, 100

...

Agent  2 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 100

Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  1 Visited :  [1000, 11.180339887498949, 1000, 9.219544457292887, 1000, 1000, 6.708203932499369, 7.0710678118654755, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000]
Agent  2 Completed :  [10

Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000,

Agent  0 Visited :  [1000, 1000, 1000, 9.433981132056603, 1000, 1000, 10.816653826391969]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  2 Completed :  [1000, 1000, 1000, 1

Agent  1 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000]
Agent  1 Completed :  [1000, 1000, 1000]
Agent  0 Visited :  [1000, 1000, 1000, 5.830951894845301, 6.4031242374328485, 6.324555320336759, 5.656854249492381, 1000, 1000, 5.830951894845301, 8.06225774829855]
Agent  1 Completed :  [1000, 1000, 1000]
Agent  0 Visited :  [1000, 1000, 1000, 1000, 6.4031242374328485, 6.324555320336759, 1000, 1000, 1000, 5.830951894845301, 8.06225774829855]
Agent  1 Completed :  [1000, 1000, 1000]
Agent  0 Visited :  [1000, 1000, 1000, 1000, 6.4031242374328485, 1000, 1000, 1000, 1000, 1000, 8.06225774829855]
Agent  1 Completed :  [1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Agent  0 Completed :  [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,

In [7]:
import time
ts = str(time.time())


for i in range(len(show_it)):
    create_output_files(show_it[i], "Eco_"+ts+"_UAV"+str(i)+".txt")
    create_output_files(rewards[i], "Eco_"+ts+"_UAV"+str(i)+"_R"+".txt")
    
create_output_files(rewards[i], "Eco_"+ts+"_NFZ"+".txt")